# Crawler de Contatos dos Grupos WhatsApp

Este notebook busca todos os contatos dos grupos do WhatsApp usando a Evolution API e salva em Excel e CSV.

## Estratégia Otimizada (v2)

A versão anterior usava `fetchAllGroups` que sincroniza com o WhatsApp e causa timeout com muitos grupos.

**Nova estratégia:**
1. Usa `/chat/findChats` para obter lista de chats do **cache local** (muito mais rápido)
2. Filtra apenas grupos (JIDs terminando em `@g.us`)
3. Busca participantes de cada grupo individualmente com **delays anti-ban**

**Configurações anti-ban:**
- Delay de 2-5 segundos entre requisições de participantes
- Timeout de 30s por grupo (fallback gracioso)

In [6]:
import os
import asyncio
import httpx
import pandas as pd
from pathlib import Path
from typing import List, Dict
from datetime import datetime

# Verificar e instalar openpyxl se necessário (para salvar Excel)
try:
    import openpyxl
except ImportError:
    print("📦 Instalando openpyxl...")
    import subprocess
    subprocess.check_call(["pip", "install", "openpyxl"])
    import openpyxl
    print("✅ openpyxl instalado")

# Configurações da Evolution API
EVOLUTION_URL = os.getenv("EVOLUTION_API_URL", "http://localhost:8080")
EVOLUTION_API_KEY = os.getenv("EVOLUTION_API_KEY")
INSTANCE = os.getenv("EVOLUTION_INSTANCE", "Revoluna")

if not EVOLUTION_API_KEY:
    raise ValueError("EVOLUTION_API_KEY não configurada. Configure no .env ou variáveis de ambiente.")

print(f"✅ Evolution API URL: {EVOLUTION_URL}")
print(f"✅ Instance: {INSTANCE}")
print(f"✅ API Key: {EVOLUTION_API_KEY[:10]}..." if len(EVOLUTION_API_KEY) > 10 else "✅ API Key configurada")


✅ Evolution API URL: http://localhost:8080
✅ Instance: Revoluna
✅ API Key: c24ecc525f...


In [ ]:
class EvolutionContactsCrawler:
    """Cliente para buscar contatos dos grupos via Evolution API.
    
    Estratégia otimizada:
    1. Usa /chat/findChats para obter lista de chats do cache local (rápido)
    2. Filtra apenas grupos (@g.us)
    3. Busca participantes por grupo individualmente com delays anti-ban
    """
    
    def __init__(self):
        self.base_url = EVOLUTION_URL.rstrip("/")
        self.api_key = EVOLUTION_API_KEY
        self.instance = INSTANCE
        self.headers = {"apikey": self.api_key}
    
    async def testar_conexao(self) -> bool:
        """Testa se a conexão com a Evolution API está funcionando."""
        url = f"{self.base_url}/instance/fetchInstances"
        
        async with httpx.AsyncClient() as client:
            try:
                response = await client.get(url, headers=self.headers, timeout=10.0)
                response.raise_for_status()
                print(f"✅ Conexão com Evolution API OK (Status: {response.status_code})")
                return True
            except httpx.ConnectError as e:
                print(f"❌ Erro de conexão: Não foi possível conectar em {self.base_url}")
                print(f"   Verifique se a Evolution API está rodando e a URL está correta")
                return False
            except httpx.HTTPStatusError as e:
                print(f"❌ Erro HTTP {e.response.status_code}: {e.response.text[:200]}")
                return False
            except Exception as e:
                print(f"❌ Erro ao testar conexão: {type(e).__name__}: {e}")
                return False
    
    async def buscar_chats_cached(self) -> List[Dict]:
        """
        Busca todos os chats do cache local via /chat/findChats.
        Muito mais rápido que fetchAllGroups pois não sincroniza com WhatsApp.
        """
        url = f"{self.base_url}/chat/findChats/{self.instance}"
        
        print(f"🔗 URL: {url}")
        
        async with httpx.AsyncClient(timeout=60.0) as client:
            try:
                response = await client.get(url, headers=self.headers)
                response.raise_for_status()
                chats = response.json()
                
                if isinstance(chats, list):
                    print(f"✅ {len(chats)} chats encontrados no cache")
                    return chats
                elif isinstance(chats, dict):
                    # Pode retornar como {"chats": [...]} ou {"data": [...]}
                    chats_list = chats.get("chats") or chats.get("data") or []
                    print(f"✅ {len(chats_list)} chats encontrados no cache")
                    return chats_list
                else:
                    print(f"⚠️ Formato inesperado: {type(chats)}")
                    return []
                    
            except Exception as e:
                print(f"⚠️ Erro ao buscar chats: {type(e).__name__}: {e}")
                return []
    
    async def filtrar_grupos(self, chats: List[Dict]) -> List[Dict]:
        """
        Filtra apenas os grupos (JID termina com @g.us).
        """
        grupos = []
        for chat in chats:
            # O chat pode ter 'id' ou 'remoteJid'
            jid = chat.get("id") or chat.get("remoteJid") or ""
            if jid.endswith("@g.us"):
                grupos.append({
                    "id": jid,
                    "nome": chat.get("name") or chat.get("subject") or chat.get("pushName") or "Sem nome",
                    "dados_originais": chat
                })
        
        print(f"📋 {len(grupos)} grupos encontrados (de {len(chats)} chats)")
        return grupos
    
    async def buscar_participantes_grupo(self, grupo_jid: str) -> List[Dict]:
        """
        Busca participantes de um grupo específico.
        Usa POST /group/participants/{instance} com groupJid no body.
        """
        url = f"{self.base_url}/group/participants/{self.instance}"
        body = {"groupJid": grupo_jid}
        
        async with httpx.AsyncClient(timeout=30.0) as client:
            try:
                # Tentar POST primeiro (mais confiável)
                response = await client.post(url, headers=self.headers, json=body)
                
                if response.status_code == 404 or response.status_code == 405:
                    # Fallback para GET com query param
                    url_get = f"{self.base_url}/group/participants/{self.instance}?groupJid={grupo_jid}"
                    response = await client.get(url_get, headers=self.headers)
                
                response.raise_for_status()
                data = response.json()
                
                # A resposta pode ser lista direta ou objeto com 'participants'
                if isinstance(data, list):
                    return data
                elif isinstance(data, dict):
                    return data.get("participants") or data.get("data") or []
                return []
                
            except Exception as e:
                print(f"      ⚠️ Erro: {type(e).__name__}")
                return []
    
    async def listar_grupos_fallback(self) -> List[Dict]:
        """
        Fallback: tenta fetchAllGroups com timeout muito longo.
        Só usar se buscar_chats_cached não funcionar.
        """
        url = f"{self.base_url}/group/fetchAllGroups/{self.instance}?getParticipants=false"
        
        print(f"⚠️ Usando fallback fetchAllGroups (pode demorar)...")
        print(f"🔗 URL: {url}")
        
        # Timeout de 10 minutos - a API pode ser muito lenta
        async with httpx.AsyncClient(timeout=600.0) as client:
            try:
                response = await client.get(url, headers=self.headers)
                response.raise_for_status()
                grupos = response.json()
                
                if isinstance(grupos, list):
                    return grupos
                elif isinstance(grupos, dict):
                    return grupos.get("groups") or grupos.get("data") or []
                return []
                
            except httpx.ReadTimeout:
                print(f"❌ Timeout após 10 minutos - API muito lenta")
                print(f"💡 Verifique se há muitos grupos ou se a Evolution API está sobrecarregada")
                return []
            except Exception as e:
                print(f"❌ Erro: {type(e).__name__}: {e}")
                return []

# Criar instância do crawler
crawler = EvolutionContactsCrawler()
print("✅ Crawler inicializado")

# Testar conexão primeiro
print("\n🔍 Testando conexão com Evolution API...")
conexao_ok = await crawler.testar_conexao()

# Verificar instâncias disponíveis
if conexao_ok:
    print("\n🔍 Verificando instâncias disponíveis...")
    url = f"{crawler.base_url}/instance/fetchInstances"
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=crawler.headers, timeout=10.0)
            response.raise_for_status()
            instances = response.json()
            
            if isinstance(instances, list):
                instance_names = [inst.get('name', '') for inst in instances if isinstance(inst, dict)]
                print(f"   Instâncias disponíveis: {instance_names}")
                
                if INSTANCE not in instance_names:
                    print(f"\n⚠️ ATENÇÃO: A instância '{INSTANCE}' não foi encontrada!")
                else:
                    instancia_info = next((inst for inst in instances if inst.get('name') == INSTANCE), None)
                    if instancia_info:
                        status = instancia_info.get('connectionStatus', 'unknown')
                        print(f"\n✅ Instância '{INSTANCE}' encontrada!")
                        print(f"   Status da conexão: {status}")
                        if status != 'open':
                            print(f"   ⚠️ A instância não está conectada (status: {status})")
        except Exception as e:
            print(f"⚠️ Não foi possível verificar instâncias: {e}")

In [ ]:
import random

# Configurações anti-ban
DELAY_MIN = 2  # segundos entre requisições de participantes
DELAY_MAX = 5  # segundos entre requisições de participantes

async def extrair_contatos_todos_grupos() -> List[Dict]:
    """
    Extrai todos os contatos de todos os grupos.
    
    Estratégia otimizada:
    1. Busca chats do cache local (rápido)
    2. Filtra grupos
    3. Busca participantes de cada grupo com delays anti-ban
    """
    
    # 1. Buscar chats do cache (muito mais rápido que fetchAllGroups)
    print("🔍 Buscando chats do cache local...")
    chats = await crawler.buscar_chats_cached()
    
    # 2. Se não encontrou chats, tentar fallback
    if not chats:
        print("\n⚠️ Cache vazio ou não disponível")
        print("💡 Tentando endpoint alternativo (findGroups)...")
        
        # Tentar endpoint de grupos direto
        grupos = await crawler.listar_grupos_fallback()
        
        if not grupos:
            print("\n❌ Não foi possível obter lista de grupos")
            print("💡 Possíveis causas:")
            print("   • A instância não está conectada ao WhatsApp")
            print("   • A instância não participa de nenhum grupo")
            print("   • A Evolution API está sobrecarregada")
            return []
        
        # Grupos do fallback já vêm com dados básicos
        grupos_filtrados = [
            {"id": g.get("id", ""), "nome": g.get("subject", "Sem nome")}
            for g in grupos
            if g.get("id", "").endswith("@g.us")
        ]
    else:
        # 3. Filtrar apenas grupos
        grupos_filtrados = await crawler.filtrar_grupos(chats)
    
    if not grupos_filtrados:
        print("\n⚠️ Nenhum grupo encontrado!")
        return []
    
    print(f"\n✅ {len(grupos_filtrados)} grupos para processar")
    
    # Listar grupos encontrados
    print("\n📋 Grupos encontrados:")
    for i, grupo in enumerate(grupos_filtrados[:20], 1):  # Mostrar até 20
        print(f"  {i}. {grupo['nome'][:50]}")
    if len(grupos_filtrados) > 20:
        print(f"  ... e mais {len(grupos_filtrados) - 20} grupos")
    
    # 4. Buscar participantes de cada grupo com delays
    print("\n🔍 Buscando participantes de cada grupo...")
    print(f"   ⏱️ Delay entre requisições: {DELAY_MIN}-{DELAY_MAX}s (anti-ban)\n")
    
    todos_contatos = []
    grupos_com_erro = 0
    
    for idx, grupo in enumerate(grupos_filtrados, 1):
        grupo_jid = grupo["id"]
        grupo_nome = grupo["nome"]
        
        print(f"[{idx}/{len(grupos_filtrados)}] {grupo_nome[:40]}...", end=" ")
        
        participantes = await crawler.buscar_participantes_grupo(grupo_jid)
        
        if participantes:
            print(f"✅ {len(participantes)} participantes")
            
            for participante in participantes:
                # Extrair informações do participante
                participante_id = participante.get("id", "")
                push_name = participante.get("pushName", "")
                name = participante.get("name", "")
                
                # Usar pushName se disponível, senão name
                nome_contato = push_name or name or ""
                
                # Extrair número de telefone do JID
                telefone = ""
                if participante_id and "@s.whatsapp.net" in participante_id:
                    telefone = participante_id.split("@")[0]
                
                if telefone:  # Só adicionar se tem telefone
                    todos_contatos.append({
                        "Nome": nome_contato,
                        "Numero_Telefone": telefone,
                        "Grupo_Origem": grupo_nome,
                        "Grupo_JID": grupo_jid
                    })
        else:
            print(f"⚠️ Sem participantes (ou erro)")
            grupos_com_erro += 1
        
        # Delay anti-ban (exceto no último)
        if idx < len(grupos_filtrados):
            delay = random.uniform(DELAY_MIN, DELAY_MAX)
            await asyncio.sleep(delay)
    
    # Resumo
    print(f"\n" + "="*50)
    print(f"📊 RESUMO")
    print(f"="*50)
    print(f"Grupos processados: {len(grupos_filtrados)}")
    print(f"Grupos com erro: {grupos_com_erro}")
    print(f"Total de contatos coletados: {len(todos_contatos)}")
    
    return todos_contatos

# Executar extração
contatos = await extrair_contatos_todos_grupos()

In [ ]:
# Criar DataFrame
df = pd.DataFrame(contatos)

if df.empty:
    print("⚠️ DataFrame vazio - nenhum contato foi coletado")
else:
    # Remover duplicatas (mesmo número em grupos diferentes)
    print(f"📊 Total de registros: {len(df)}")
    print(f"📊 Contatos únicos (por número): {df['Numero_Telefone'].nunique()}")

    # Mostrar estatísticas
    print("\n📈 Estatísticas:")
    print(f"  • Grupos processados: {df['Grupo_Origem'].nunique()}")
    print(f"  • Contatos com nome: {df[df['Nome'] != '']['Nome'].count()}")
    print(f"  • Contatos sem nome: {df[df['Nome'] == '']['Nome'].count()}")

    # Mostrar preview
    print("\n👀 Preview dos dados:")
    print(df.head(10))

In [ ]:
# Verificar se temos contatos antes de salvar
if not contatos or len(contatos) == 0:
    print("⚠️ Nenhum contato para salvar")
else:
    # Criar diretório de saída se não existir
    output_dir = Path("../../data/contatos_grupos")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Gerar nome do arquivo com timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_file = output_dir / f"contatos_grupos_{timestamp}.csv"
    excel_file = output_dir / f"contatos_grupos_{timestamp}.xlsx"

    # Salvar CSV
    df.to_csv(csv_file, index=False, encoding='utf-8-sig')
    print(f"✅ CSV salvo: {csv_file}")

    # Salvar Excel
    df.to_excel(excel_file, index=False, engine='openpyxl')
    print(f"✅ Excel salvo: {excel_file}")

    print(f"\n📁 Arquivos salvos em: {output_dir.absolute()}")
    print(f"   • {csv_file.name}")
    print(f"   • {excel_file.name}")

In [ ]:
# Análise adicional (apenas se temos dados)
if not df.empty:
    print("📊 Contatos por grupo:")
    contatos_por_grupo = df.groupby('Grupo_Origem').size().sort_values(ascending=False)
    print(contatos_por_grupo.head(20))
    if len(contatos_por_grupo) > 20:
        print(f"... e mais {len(contatos_por_grupo) - 20} grupos")

    # Contatos que aparecem em múltiplos grupos
    print("\n👥 Contatos em múltiplos grupos:")
    contatos_multiplos = df.groupby('Numero_Telefone').agg({
        'Nome': 'first',
        'Grupo_Origem': lambda x: ', '.join(x.unique()),
        'Numero_Telefone': 'count'
    }).rename(columns={'Numero_Telefone': 'Qtd_Grupos'})
    contatos_multiplos = contatos_multiplos[contatos_multiplos['Qtd_Grupos'] > 1].sort_values('Qtd_Grupos', ascending=False)
    print(f"Total: {len(contatos_multiplos)} contatos aparecem em mais de um grupo")
    if not contatos_multiplos.empty:
        print(contatos_multiplos.head(20))
else:
    print("⚠️ Sem dados para análise")